# 封装-按图例聚类-综合图


功能：

1. 自动识别图例
2. 将三通道png对应图例的颜色替换为单通道png评级
3. 将png转为tif
4. 对tif进行粗投影

In [1]:
## 图例自动识别
import cv2
import numpy as np

def legend(legend_name):
    # 读取图片
    img = cv2.imread(f"../rawdata/风险图例/{legend_name}.png")
    height, width = img.shape[:2]
    # 计算横向中心和纵向等分点的坐标
    x = width // 2 # 横向中心
    if legend_name in ['11.森林火灾','12.草原火灾']: y_list = [height * i // 18 for i in range(1, 18, 2)]
    else: y_list = [height * i // 20 for i in range(1, 20, 2)] # 纵向1/20, 3/20, ..., 19/20

    # 提取每个节点的BGR值生成ranks, colors
    colors = []
    for y in y_list:
        # 计算方格的左上角和右下角的坐标
        x1 = max(0, x - 10) # 防止越界
        y1 = max(0, y - 5)
        x2 = min(width - 1, x + 9)
        y2 = min(height - 1, y + 4)
        # 提取方格内的像素值
        patch = img[y1:y2+1, x1:x2+1]
        # 计算方格内的像素值的平均值，得到BGR均值
        bgr = np.mean(patch, axis=(0, 1))
        colors.append(bgr)
    if legend_name in ['11.森林火灾','12.草原火灾']: colors = colors + [[233,233,233]]
    ranks = list(range(1,11))
    
    # 输出结果
    return ranks, colors

In [3]:
# 导出图例
import os
import pandas as pd

name_lst, rank_lst, b_lst, g_lst, r_lst = [], [], [], [], []
for image_name in [name[:-4] for name in os.listdir('../rawdata/风险图集') if name[-4:] == '.png']:
    ranks,colors = legend(image_name)
    name_lst += [image_name]*len(ranks)
    rank_lst += ranks
    b_lst += [color[0] for color in colors]
    g_lst += [color[1] for color in colors]
    r_lst += [color[2] for color in colors]

legend_df = pd.DataFrame({'name':name_lst, 'rank':rank_lst, 'b':b_lst, 'g':g_lst, 'r':r_lst})
legend_df.to_csv('../rawdata/图例_f.csv',encoding='utf-8', index=False)

In [86]:
## 基于图例的图片转tif
import cv2
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt

def png_to_tif(image_name):
    ## 存取设置
    old_png_file = f"../rawdata/风险图集/{image_name}.png"
    new_png_file = f"../workingdata/{image_name}.png"
    tif_file = f"../outputs/{image_name}.tif"
    
    ## 读取old_png图片颜色
    # 读取图片
    image = cv2.imread(old_png_file)
    # image = cv2.fastNlMeansDenoisingColored(image, None, 5, 2, 7, 15)
    data = image.reshape((-1, 3))
    # 读取图例和索引
    ranks = np.append(legend_df[legend_df['name']==image_name]['rank'].to_numpy(),-1)
    colors = np.append(legend_df[legend_df['name']==image_name].loc[:,['b','g','r']].to_numpy(),[[110,110,110]],axis=0)

    ## 按图例聚类转new_png
    # 计算图片中每个像素值和颜色列表中每个颜色的欧氏距离，并获取最近颜色
    distances = np.max(np.abs(data[:, np.newaxis] - colors), axis=2)
    indices = np.argmin(distances, axis=1)
    # 根据索引替换原来的像素值
    new_image = ranks[indices]
    new_image = new_image.reshape(image.shape[:2])
    # 去除异常要素
    mask = new_image == -1
    new_image = new_image.astype(np.uint8) # 转换为uint8类型
    mask = mask.astype(np.uint8) # 转换为uint8类型
    new_image = cv2.inpaint(new_image,mask,3,cv2.INPAINT_NS)
    # 保存图片
    cv2.imwrite(new_png_file, new_image)

    ## new_png转tif
    # 打开png文件
    png_ds = gdal.Open(new_png_file)
    ysize, xsize = new_image.shape[:2]
    # 转换为tif文件
    tif_ds = gdal.Translate(tif_file, png_ds, format="GTiff")
    
    ## 投影tif文件
    srs = osr.SpatialReference() # 创建空的SpatialReference对象
    srs.ImportFromEPSG(32648)
    proj = srs.ExportToWkt()
    # 粗投影
    xmin, ymin = -2225850,1749624 # 左下角平面坐标
    xmax, ymax = 3267843,6173492 # 右上角平面坐标list
    xres = (xmax - xmin) / xsize # 横向分辨率
    yres = (ymax - ymin) / ysize # 纵向分辨率
    tif_ds.SetProjection(proj) # 设置投影类型
    tif_ds.SetGeoTransform([xmin, xres, 0, ymax, 0, -yres]) # 设置范围
    # 精配准
    # gcp_lst = [
    #     gdal.GCP(1830831.705, 5529138.383, 0, 1284.14, 127.801),
    #     gdal.GCP(998192.186, 4154895.379, 0, 1109.594, 496.089),
    #     gdal.GCP(-315864.18, 2409175.718, 0, 831.469, 981.185),
    #     gdal.GCP(-1189853.39, 4947446.517, 0, 543.009, 366.456),
    #     gdal.GCP(-1610510.422, 5874044.196, 0, 396.31, 163.703),
    #     gdal.GCP(-3337578.725, 5087943.408, 0, 50.319, 431.215),
    #     gdal.GCP(478261.212, 5527136.724, 0, 936.828, 160.954),
    #     gdal.GCP(-3027115.268, 5421381.964, 0, 103.142, 344.411),
    #     gdal.GCP(-2338540.013, 3348151.907, 0, 320.865, 792.632),
    #     gdal.GCP(-1092279.572, 2423867.539, 0, 633.67, 993.725),
    #     gdal.GCP(706252.866, 5835242.114, 0, 987.584, 77.013)
    # ]
    # tif_ds.SetGCPs(gcp_lst, proj, ) # 设置GCPs和投影类型
    # 关闭Dataset对象
    png_ds = None
    tif_ds = None

In [85]:
png_to_tif("综合风险图")

ERROR 1: PROJ: proj_create_from_database: Open of /home/dsw/.conda/envs/geo/share/proj failed
